In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = "sk-Vn416fJ5l5gRRNoQnaIIMEbepS7T_EyaxWv8d3eTqIT3BlbkFJfReJ5s0y1hv82SWyit1aaKwM_PkX_Fx6c4GZjvsqoA"

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

from tos.ontology.utdo import *

structured_llm = llm.with_structured_output(Permission)

output = structured_llm.invoke("""
You are an expert Knowledge Engineer and you are working on a project to build a knowledge graph.
You are required to generate a Policy object that represents the input term.

Stick to the specified format and ouput valid JSON.

Format

Term: '''
GitHub has the right to suspend or terminate your access to all or any part of the Website at any time, with or without cause, with or without notice, effective immediately. GitHub reserves the right to refuse service to anyone for any reason at any time.
'''
""")


NameError: name 'Permission' is not defined

In [ ]:
output

Permission(assignee=Party(id=1, name='GitHub', type='PROVIDER'), action=Action(target=Asset(id=1, name='Website', type='SERVICE')), constraints=[Constraint(type=['UNILATERALLY', 'NO_PRIOR_NOTICE_REQUIRED'])])

In [35]:
import os

os.environ["OPENAI_API_KEY"] = "sk-Vn416fJ5l5gRRNoQnaIIMEbepS7T_EyaxWv8d3eTqIT3BlbkFJfReJ5s0y1hv82SWyit1aaKwM_PkX_Fx6c4GZjvsqoA"

from enum import Enum
from typing import Union

from pydantic import BaseModel

import openai
from openai import OpenAI

from tos.ontology.utdo import *


client = OpenAI()

SYSTEM_PROMT = """
You are an expert Knowledge Engineer and you are working on a project to build a knowledge graphs.
You are required to generate a Policy object that represents the input term. Stick to the specified format and ouput valid JSON.
Make sure the ActionType is one of the following: DELETE, MODIFY, SUSPEND, CONSENT
Make sure the deontic modalities is correct:
 - IS_OBLIGATED_TO when the subject is obligated to do something.
 - HAS_RIGHT_TO when the subject is permitted to do something.
 - IS_PROHIBITED_TO when the subject is not permitted to do something.
"""

USER_PROMPT = """
Input term: '''GitHub has the right to suspend or terminate your access to all or any part of the Website at any time, with or without cause, with or without notice, effective immediately. GitHub reserves the right to refuse service to anyone for any reason at any time.'''
"""

SYSTEM_PROMT = """
You are an expert Ontology Engineer. Follow this steps carefully and you will receive a 5000$ bonus.
Extract all the ActionTarget objects from the input term.
An ActionTarget is a noun phrase that represents the target of an action.
Include ONLY specific targets, not general ones.
Assign the type correctly. Follow this guidelines:
- USER_CONTENT: content, data or information somehow is related to the user.
- SERVICE: a service that the user can access or use.
- CONTRACT: the contract itself.
"""

USER_PROMPT = """
Input term: '''GitHub has the right to suspend or terminate your access to all or any part of the Website at any time, with or without cause, with or without notice, effective immediately. GitHub reserves the right to refuse service to anyone for any reason at any time.'''
"""

completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": SYSTEM_PROMT.strip(),
        },
        {
            "role": "user",
            "content": USER_PROMPT
        },
    ],
    tools=[
        openai.pydantic_function_tool(ActionTargetList),
    ],
)

print(completion.choices[0].message.tool_calls[0].function.parsed_arguments)

targets=[ActionTarget(id=1, name='your access to all or any part of the Website', type='SERVICE'), ActionTarget(id=2, name='service', type='SERVICE')]


In [3]:
print(completion.choices)

[ParsedChoice[NoneType](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[NoneType](content='```json\n{\n  "Policy": {\n    "Subject": "GitHub",\n    "Action": "suspend or terminate access",\n    "Asset": "Website",\n    "Modality": "HAS_RIGHT_TO",\n    "Conditions": "at any time, with or without cause, with or without notice, effective immediately"\n  }\n}\n```', refusal=None, role='assistant', function_call=None, tool_calls=[], parsed=None))]
